In [1]:
import os
import pandas as pd
from datetime import datetime

In [2]:
path = os.getcwd()

In [3]:
csv_files_m = []
for files in os.listdir(os.path.join(path,'Datos','Meteorologicos')):
    if files[-3:] == 'csv':
        csv_files_m.append(files)
    else:
        pass
csv_files_c = []
for files in os.listdir(os.path.join(path,'Datos','Contaminantes')):
    if files[-3:] == 'csv':
        csv_files_c.append(files)
    else:
        pass

In [4]:
met_data = []
for file in csv_files_m:
    file = str(file)
    aux = pd.read_csv(os.path.join('Datos','Meteorologicos',file),sep = ';',usecols = [0,1,2])

    s_date = datetime.strptime(str(aux.iloc[0,0])+' '+str(aux.iloc[0,1]).replace("00",""), '%y%m%d %H')
    e_date = datetime.strptime(str(aux.iloc[-1,0])+' '+str(aux.iloc[-1,1]).replace("00",""), '%y%m%d %H')

    index = pd.date_range(start = s_date, end = e_date, freq = 'H')

    aux = aux.set_index(index)
    aux = aux.drop(columns = aux.columns[0:2])
    aux.columns = [file[4:-4]]

    met_data.append(aux)
    
cont_data = []
for file in csv_files_c:
    file = str(file)
    aux = pd.read_csv(os.path.join('Datos','Contaminantes',file),sep = ';',usecols = [0,1,2,3])
    s_date = datetime.strptime(str(aux.iloc[0,0])+' '+str(aux.iloc[0,1]).replace("00",""), '%y%m%d %H')
    e_date = datetime.strptime(str(aux.iloc[-1,0])+' '+str(aux.iloc[-1,1]).replace("00",""), '%y%m%d %H')

    index = pd.date_range(start = s_date, end = e_date, freq = 'H')

    aux = aux.set_index(index)
    aux = aux.drop(columns = aux.columns[0:2])

    aux_isna = aux.isna()
    for i in range(len(aux)):
        if aux_isna.iloc[i,0] == True and aux_isna.iloc[i,1] == False:
            aux.iloc[i,0] = aux.iloc[i,1]
    aux = aux.drop(columns = aux.columns[1])
    aux.columns = [file[4:-4]]

    cont_data.append(aux)

In [5]:
for i, data in enumerate(met_data):
    met_data[i] = data.loc['2018':'2022-08-31']
for i, data in enumerate(cont_data):    
    cont_data[i] = data.loc['2018':'2022-08-31']

In [6]:
met_df = pd.concat(met_data,axis=1)
cont_df = pd.concat(cont_data,axis=1)

In [7]:
data_df = pd.concat([met_df, cont_df], axis = 1) 

In [8]:
data_df.columns

Index(['Presion', 'Temperatura', 'HR', 'RapViento', 'O3', 'CO', 'PM25', 'PM10',
       'SO2'],
      dtype='object')

In [9]:
for j, col in enumerate(data_df.columns):
    for i,value in enumerate(data_df.iloc[:,j].values):
        aux = str(value)
        data_df.iloc[i,j] = aux.replace(",",".")
    
    data_df[col] = pd.to_numeric(data_df[col],errors='coerce')

In [15]:
doy = [int(ind.strftime('%j')) for ind in data_df.index]
dow = [int(ind.strftime('%w')) for ind in data_df.index]

for i,value in enumerate(dow):
    if value == 0:
        dow[i] = 7

In [16]:
data_df['DoY'] = doy
data_df['DoW'] = dow

In [17]:
data_df.to_csv(os.path.join(path,'Datos','Data_Coyhaique.csv'))